<a href="https://colab.research.google.com/github/swapnildahare/RAG_Sanskrit_Swapnil_Dahare/blob/main/Sanksrit_RAG_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers faiss-cpu transformers python-docx indic-transliteration pdfminer.six


In [ ]:
import os
import re
import json
import numpy as np
import faiss
from docx import Document
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from indic_transliteration.sanscript import transliterate, DEVANAGARI, IAST


In [ ]:
def read_docx(path):
    doc = Document(path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)

In [ ]:
doc_path = "/content/Rag-docs (1).docx"
raw_text = read_docx(doc_path)

print("DOCUMENT LOADED")
print(raw_text[:500])

DOCUMENT LOADED
मूर्खभृत्यस्य

"अरे शंखनाद, गच्छापणम्, शर्कराम् आनय ।" इति स्वभृत्यम् शंखनादम् गोवर्धनदासः आदिशति । ततः शंखनादः आपणम् गच्छति, शर्कराम् जीर्णे वस्त्रे न्यस्यति च । तस्मात् जीर्णवस्त्रात् मार्गे एव सर्वापि शर्करा स्त्रवति । ततः गोवर्धनदासः कोपेन शंखनादम् वदति, "अरे मूढ, कुत्रास्ति शर्करा ? शर्करादिकम् एवम् जीर्णेन वस्त्रेण न एवानयन्ति कदापि । इतःपरम् किमपि वस्तुजातम् दृढायाम् सन्चिकायाम् निक्षिप्य आनय च " इति । अत्रान्तरे गोवर्धनदासस्य पुत्रः "श्वानशावकम् आनय" इति शंखनादम् आदिशति । आज्ञापालकः शंखन


In [ ]:
def normalize_text(text):
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

norm_text = normalize_text(raw_text)

In [ ]:
CHUNK_SIZE = 800
STRIDE = 400

def chunk_text(text, doc_id="sanskrit_doc"):
    chunks = []
    start = 0
    end = len(text)
    idx = 0

In [ ]:
def chunk_text(text, doc_id="sanskrit_doc"):
    chunks = []
    start = 0
    end = len(text)
    idx = 0
    while start < end:
        chunk = text[start: start + CHUNK_SIZE]

        if any('\u0900' <= c <= '\u097F' for c in chunk):
            deva = chunk
            try:
                translit = transliterate(chunk, DEVANAGARI, IAST)
            except:
                translit = chunk
        else:
            translit = chunk
            try:
                deva = transliterate(chunk, IAST, DEVANAGARI)
            except:
                deva = chunk

        chunks.append({
            "chunk_id": f"{doc_id}_{idx}",
            "text_deva": deva,
            "text_translit": translit
        })

        idx += 1
        start += STRIDE

    return chunks

In [ ]:
chunks = chunk_text(norm_text)
print("Total chunks:", len(chunks))

Total chunks: 23


In [ ]:
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

corpus_embeddings = embedding_model.encode(
    [c["text_deva"] for c in chunks],
    convert_to_numpy=True,
    show_progress_bar=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
faiss.normalize_L2(corpus_embeddings)
dim = corpus_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(corpus_embeddings)

print("FAISS index built")

FAISS index built


In [ ]:
def is_devanagari(text):
    return any("\u0900" <= ch <= "\u097F" for ch in text)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
generator = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")


model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def retrieve(query, top_k=5):
    if not is_devanagari(query):
        query = transliterate(query, IAST, DEVANAGARI)

    q_emb = embedding_model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)

    D, I = index.search(q_emb, top_k)

    results = []
    for score, idx in zip(D[0], I[0]):
        results.append({
            "score": float(score),
            "chunk": chunks[idx]["text_deva"],
            "id": chunks[idx]["chunk_id"]
        })

    return results

In [ ]:
def build_prompt(query, retrieved):
    context = ""
    for r in retrieved:
        context += f"[{r['id']}]: {r['chunk']}\n\n"

    prompt = (
        "Answer the question in Sanskrit using ONLY the context.\n"
        "If not found, say 'न विद्यते'.\n\n"
        f"Context:\n{context}\n"
        f"Question:\n{query}\n\nAnswer:"
    )
    return prompt

In [ ]:
def generate_answer(query):
    retrieved = retrieve(query, top_k=5)
    prompt = build_prompt(query, retrieved)

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = generator.generate(inputs["input_ids"], max_length=200)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer, retrieved

In [ ]:
##example for test
query = "कः मूर्खभृत्यः ?"
answer, context = generate_answer(query)

print("🔹 Answer:\n", answer)
print("\n🔹 Retrieved chunks:")
for c in context:
    print(c["id"], "→", c["score"])
    print(c["chunk"][:150], "...\n")

Token indices sequence length is longer than the specified maximum sequence length for this model (1223 > 512). Running this sequence through the model will result in indexing errors


🔹 Answer:
  

🔹 Retrieved chunks:
sanskrit_doc_0 → 0.49122685194015503
मूर्खभृत्यस्य "अरे शंखनाद, गच्छापणम्, शर्कराम् आनय ।" इति स्वभृत्यम् शंखनादम् गोवर्धनदासः आदिशति । ततः शंखनादः आपणम् गच्छति, शर्कराम् जीर्णे वस्त्रे न ...

sanskrit_doc_10 → 0.4661385715007782
याः चार्तुयम् आसीत् चित्रपुरम् नाम किमपि नगरं श्रीपर्वतस्य समीपे । "पर्वतस्य शिखरप्रदेशे घण्टाकर्णः नाम राक्षसः प्रतिवसती" ति जनप्रवादः अवर्तत् । अथैक ...

sanskrit_doc_11 → 0.4353455603122711
ण्टाणादं च अकुर्वन् । चित्रपुरस्थाः नागरिकाः वारं वारं पर्वतशिखरप्रदेशात् घण्टानादमाकर्णयन् । भयाकुलाः ते अचिन्तयन् "नुनं शिखरप्रदेशे घण्टकर्णः नाम रा ...

sanskrit_doc_22 → 0.4092165231704712
गुअगे सो wएल्ल्, थेन् मेएतिन्ग् wइथ् थे स्छोलर्स् स्हल्ल् चेर्तैन्ल्य् लेअद् तो म्य् देफ़ेअत्। थेन् हे ओर्देरेद् कलिदस, इ रेअल्ल्य् दो नोत् wअन्त् तो ग ...

sanskrit_doc_15 → 0.3951345980167389
तः साहाय्यम् न आवश्यकम् । देवः अस्ति । सः एव साहाय्यम् करोति" इति। सः अपि गतवान् । इदानिम वृष्टिः अधिका अभवत् । जलम् तस्य कन्ठपर्यंतम् आगतम् । तदा पुन ...



In [ ]:
query ="घोषितं कदाचित् भोजराज्ञा"
answer, context = generate_answer(query)

print("🔹 Answer:\n", answer)
print("\n🔹 Retrieved chunks:")
for c in context:
    print(c["id"], "→", c["score"])
    print(c["chunk"][:150], "...\n")

🔹 Answer:
                                                                                                    

🔹 Retrieved chunks:
sanskrit_doc_13 → 0.8038051128387451
वस्य प्रार्थनाम् करोति । "देव, कृपया मह्यं आरोग्यम् ददातु, धनम् ददातु" इति । सः किंचित् अपि प्रयत्नम् न करोति, कार्यम् न करोति । देवः साहाय्यम् करिष्य ...

sanskrit_doc_12 → 0.7983924150466919
भृतम् अतिष्ठत् । 'वानराः एव घण्टां वादयन्ति' इति सा अपश्यत् । अन्येद्युः सा वानरेभ्यः मधुराणि फलाणि अयच्छत् । यावत् ते फलभक्षणमग्नाः संजाताः, तावदेव त ...

sanskrit_doc_16 → 0.7506968975067139
 । सः देवम् पृष्टवान् "देव, अहम् भवतः परमभक्तः । यदा मम कष्टः अभवत्, भवान् किंचित् अपि साहाय्यम् न कृतवान् । भवान् द्रष्टुम् अपि न आगतवान् । किमर्थम्  ...

sanskrit_doc_11 → 0.7108046412467957
ण्टाणादं च अकुर्वन् । चित्रपुरस्थाः नागरिकाः वारं वारं पर्वतशिखरप्रदेशात् घण्टानादमाकर्णयन् । भयाकुलाः ते अचिन्तयन् "नुनं शिखरप्रदेशे घण्टकर्णः नाम रा ...

sanskrit_doc_15 → 0.7085952162742615
तः साहाय्यम् न आवश्यकम् । देवः अस्ति । सः एव साहाय्यम् करोत